The purpose of this notebook is to go through the computations to estimate Critical Pace at Gradient (CPG). There are several models for cycling to compute this, however we will (at least initially) use the simplest model. In particular, of $P(t)$ is maximum pace which can be sustained for $t$ seconds, $CP$ is critical pace, and $W'$ is the "anaerobic work capacity", then $P(t) = CP + W'/t$. Thus $t\cdot P(t) = t\cdot CP + W'$, and thus $CP$ is the slope of the line $t\cdot P(t)$, and $W'$ is the intercept. So this notebook will essentially be setting up this equation, and then applying linear regression to it.

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression

First we assume that the runs have been converted to csv. We look at the csv folder and read them in, clean up the gradient, and create the gradient_100 column.

In [2]:
def runs_to_df(INDIR, clean_gradient=True):
    ''' Returns a list of dataframes for each run. These dataframes can optionally have gradient
    spikes removed by leaving clean_gradient to True.'''
    files = os.listdir(INDIR)
    df_list = []
    for file in files:
        if file.endswith('.csv'):
            df_list.append(pd.read_csv(INDIR + file))
            
    for df in df_list:
        df['gradient'] = clean_gradient_spikes(df['gradient'])
        df['gradient_100'] = 100*df['gradient']
        
    return df_list

Next we compute the maximum speed at a time $t$. In fact, this function returns a vector with the maximum speed achieved for time interval $t$ at all gradients at which such an interval occurs. So for example, if $t=60$ seconds, then it looks for all 60 second time intervals, finds the average gradient of the interval, and computes the speed. Further computations on this vector are done in the function compute_CPG_matrix.

In [3]:
def better_max_speed_t(t, df):
    rolling_df = df[['inst_speed_meters_sec', 'gradient_100']].rolling(window=t).mean().fillna(0)
    rolling_df['rounded_gradient_100'] = rolling_df['gradient_100'].apply(np.round)
    rolling_groupby = rolling_df.groupby('rounded_gradient_100')
    
    max_speed = {int(name): max(group['inst_speed_meters_sec'].values) for name,group in rolling_groupby}
    return max_speed

This is a helper function which removes gradients over 40%

In [4]:
def clean_gradient_spikes(grad_series, replacement_grad='median', max_grad=0.4):
    bad_index = grad_series.loc[np.abs(grad_series) > max_grad].index
    grad_series.fillna(0)
    if len(bad_index) == 0:
        return grad_series
    elif replacement_grad == 'median':
        grad_series.loc[bad_index] = np.median(grad_series.values)
    elif replacement_grad == 'mean':
        grad_series.loc[bad_index] = np.mean(grad_series.values)
    elif replacement_grad == 'zero':
        grad_series.loc[bad_index] = 0
    return grad_series

Next we iterate through all time intervals the user asks for (later, in the variable $t$), and create a matrix (from an intermediate data frame) from this. This is the "max speed matrix" for that run, because the $(t,g)$ entry holds the maximum speed achieved in a time interval of $t$ with average gradient $g$. Some intermediate work is done to "fill in" those gradients which don't appear. For example, perhaps a gradient of 10% was never run for 60 seconds, but a 0% gradient was run for that time. Then there is an entry for $(60,0)$, but not for $(60,10)$. So we fill in zeros in those missing entries.

In [5]:
def compute_CPG_matrix(df_list, time_interval, gradient_interval, INDIR):
    df_list = runs_to_df(INDIR)
    
    max_speed_list_of_df = [pd.DataFrame([better_max_speed_t(t_val, df) for t_val in t]).fillna(0) for df in df_list]
    max_speed_list_of_df_updated = []

    for df in max_speed_list_of_df:
        df_data = df
        df = pd.DataFrame(0, index=t, columns=g)
        df.update(df_data)
        max_speed_list_of_df_updated.append(df)
        
    max_speed_list = [df[g].iloc[t].as_matrix() for df in max_speed_list_of_df_updated]
    CPG_matrix = np.maximum.reduce(max_speed_list)
    return CPG_matrix

Another helper function which, given a max speed matrix, desired gradient, and range over all gradients, returns the 1-dimensional vector of max speeds at that gradient over all times recorded. This is used to create the 1-dimensional CPG "slice" chart.

In [6]:
def gradient_speed_array(gradient, CPG_matrix, gradient_range):
    '''Return a numpy array showing the maximum speed achieved at a given gradient'''
    gradient_index = np.where(gradient_range == gradient)
    gradient_speed_array = CPG_matrix[:,gradient_index[0][0]]
    return gradient_speed_array

Read in data, specify time and gradient intervals, and do computations.

In [34]:
INDIR = r'data/csv/'
minutes = 10
t = np.arange(60*minutes+1)
g = np.arange(-10, 11)

df_list = runs_to_df(INDIR)
CPG_matrix = compute_CPG_matrix(df_list, t, g, INDIR)

/Users/paulsavala/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Using the ideas at the top of this notebook, we use linear regression to compute the CP and W' for each gradient. Note that this is done regardless of the amount of time a gradient was actually run at. So for instance, if we ran a 0% gradient for 5 minutes, it probably makes sense to compute CP and W' at a 0% gradient. However, if the longest we ever ran a 20% gradient for was 5 seconds, it probably *doesn't* make sense to compute these things. Yet, this function still does that. This needs to be dealt with at some point, perhaps by specifying a minimum time (a brief review of the research says that runners switch from anaerobic to aerobic around the 50 second mark, so one minute might be a good cutoff).

In [9]:
def compute_CP_Wprime(CPG_matrix, gradient, gradient_range, time_range):
    ''' This uses linear regression on the equation t*P(t) = t*CP + Wprime to compute CP and Wprime.'''
    CPG_fixed_gradient = gradient_speed_array(gradient, CPG_matrix, gradient_range)
    indices_with_nonzero_pace = np.nonzero(CPG_fixed_gradient)
    CPG_fixed_gradient_nonzero = CPG_fixed_gradient[indices_with_nonzero_pace]
    time_range_nonzero = time_range[indices_with_nonzero_pace]
    
    lr = LinearRegression()
    
    pace_times_time_gradient_fixed = np.array(CPG_fixed_gradient_nonzero*time_range_nonzero)
    input_length = len(time_range_nonzero)
    pace_times_time_gradient_fixed = pace_times_time_gradient_fixed.reshape((input_length,1))
    time_range = time_range_nonzero.reshape((input_length, 1)) # This reshaping is needed by sklearn

    lr.fit(time_range, pace_times_time_gradient_fixed)
    return (lr.coef_[0][0], lr.intercept_[0])

We use Bokeh for plotting, due to its maturity and simplicity.

In [13]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [10]:
CP_vals = []
Wprime_vals = []

for gradient in g:
    CP, Wprime = compute_CP_Wprime(CPG_matrix, gradient, g, t)
    CP_vals.append(CP)
    Wprime_vals.append(Wprime)

In [35]:
p = figure(title="CP chart", plot_height=300, plot_width=600, y_range=(0,7))
r = p.line(t[1:], gradient_speed_array(0, CPG_matrix, g)[1:], color="#2222aa", line_width=3)

In [36]:
show(p, notebook_handle=True)

In [37]:
def update(gradient):
    r.data_source.data['y'] = gradient_speed_array(gradient, CPG_matrix, g)[1:]
    push_notebook()

In [38]:
interact(update, gradient=(-10,10))

<function __main__.update>